### *IT3052E - Fundamentals of Optimization*
# **Mini Project 18 - Nurse Scheduling Problem**
#### **Techniques used**:
* Backtracking,
* Constraint Programming,
* Linear Programming,
* Local Search, and
* Meta-heuristics (Genetic Algorithm).

#### * ***Import pandas for printing solution***

In [ ]:
import pandas as pd

## 5. Genetic Algorithm

### 5.1. Import modules and libraries

In [ ]:
from deap import base
from deap import creator
from deap import tools
import random
import numpy
from GeneticAlgorithm import Elitism
from GeneticAlgorithm import Initialization

### 5.2. Read data from file

#### Read N, D, a, b and dayoff

In [ ]:
with open('SampleData/testCase1/0.txt') as file:
  N, D, a, b = [int(q) for q in file.readline().split()]
  dayoff = [[0 for d in range(D)] for n in range(N)]
  for n in range(N):
    for d in [int(h) for h in file.readline().split()]:
      if d != -1:
            dayoff[n][d-1] = 1

### 5.3. Initialize the instances

#### 5.3.1. Generate genetic constants and penalty factor for a hard-constraint violation

In [ ]:
POPULATION_SIZE = 1000  # 300
P_CROSSOVER = 0.95  # probability for crossover
P_MUTATION = 0.05  # probability for mutating an individual
MAX_GENERATIONS = 200  # 200
HALL_OF_FAME_SIZE = 30
HARD_CONSTRAINT_PENALTY = 10

#### 5.3.2. Create the instance to be used

In [ ]:
nsp = Initialization.NurseSchedulingProblem(HARD_CONSTRAINT_PENALTY, N, D, a, b, dayoff)

#### 5.3.3. Define a single objective, maximizing fitness strategy

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

#### 5.3.4. Other instances

In [ ]:
# Create the Individual class based on list:
creator.create("Individual", list, fitness=creator.FitnessMin)

# set the random seed:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

toolbox = base.Toolbox()
# Create an operator that randomly returns 0 or 1:
toolbox.register("zeroOrOne", random.randint, 0, 1)

# Create the individual operator to fill up an Individual instance:
toolbox.register(
    "individualCreator",
    tools.initRepeat,
    creator.Individual,
    toolbox.zeroOrOne,
    len(nsp),
)

# Create the population operator to generate a list of individuals:
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)

### 5.4. Initialize the genetic mechanisms

#### 5.4.1. Fitness calculation

In [ ]:
def getCost(individual):
    return (nsp.getCost(individual),)
toolbox.register("evaluate", getCost)

#### 5.4.2. Genetic operators

In [ ]:
toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=1.0 / len(nsp))

### 5.5. Run the GA flow

In [ ]:
# create initial population (generation 0):
population = toolbox.populationCreator(n=POPULATION_SIZE)

# prepare the statistics object:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", numpy.min)
stats.register("avg", numpy.mean)

# define the hall-of-fame object:
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

# perform the Genetic Algorithm flow with hof feature added:
population, logbook = Elitism.eaSimpleWithElitism(
    population,
    toolbox,
    cxpb=P_CROSSOVER,
    mutpb=P_MUTATION,
    ngen=MAX_GENERATIONS,
    stats=stats,
    halloffame=hof,
    verbose=True,
)

# print best solution found:
best = hof.items[0]

### 5.6. Print the schedule

In [ ]:
nsp.printScheduleInfo(best)

### 5.7. Optimal solution

In [ ]:
fitnesstmp = []
for n in range(N):
    tmp = 0
    for d in range(D):
        tmp += best[4*D*n + 4*d + 3]
    fitnesstmp.append(tmp)

maxNightShift = max(fitnesstmp)
print('Optimal solution - Max night shift assigned to a nurse:', maxNightShift)